In [21]:
import re
from pathlib import Path
from typing import Dict, List, Set

def parse_markdown_standings(content: str) -> Dict[str, Dict[str, List]]:
    """Parse markdown content and extract standings for each stage."""
    stages = {}
    current_stage = None
    
    # Find stage sections and their tables
    for line in content.split('\n'):
        if line.startswith('### '):
            current_stage = line.replace('### ', '').strip()
            stages[current_stage] = []
        elif current_stage and '|' in line and not line.startswith('|---'):
            if 'Team | W-L |' not in line:  # Skip header row
                parts = [p.strip() for p in line.split('|')]
                if len(parts) >= 3:
                    team = parts[1].strip()
                    wl = parts[2].strip()
                    if wl.startswith('+'):
                        wl = wl[1:]  # Remove the '+' prefix
                    wins, losses = map(int, wl.split('-'))
                    stages[current_stage].append({
                        'team': team,
                        'wins': wins,
                        'losses': losses,
                        'score': wins - losses
                    })
    
    # Sort teams in each stage by wins-losses
    for stage in stages:
        stages[stage] = sorted(
            stages[stage],
            key=lambda x: (x['wins'], -x['losses']),
            reverse=True
        )
    
    return stages

def get_advancing_teams(stage_data: List[dict], num_teams: int = 8) -> Set[str]:
    """Get the top N teams from a stage based on W-L record"""
    sorted_teams = sorted(stage_data, key=lambda x: (x['wins'], -x['losses']), reverse=True)
    return {team['team'] for team in sorted_teams[:num_teams]}

def analyze_advancement_predictions():
    # Setup paths
    current_dir = Path.cwd()
    if current_dir.name == 'analysis':
        results_dir = current_dir.parent
    else:
        results_dir = current_dir
    analysis_dir = results_dir / 'analysis'
    analysis_dir.mkdir(exist_ok=True)
    
    # Read real world results
    with open(results_dir / '0real-world' / 'README.md') as f:
        real_world_content = f.read()
    real_world_standings = parse_markdown_standings(real_world_content)
    
    # Get real world advancing teams for each stage
    real_advancing = {
        'stage1': get_advancing_teams(real_world_standings['stage1']),
        'stage2': get_advancing_teams(real_world_standings['stage2']),
        'stage3': get_advancing_teams(real_world_standings['stage3'])
    }
    
    # Get real world champion
    playoffs_data = real_world_standings['playoffs']
    real_champion = next(team['team'] for team in playoffs_data if team['wins'] == 3)
    
    # Analyze each model
    results = {}
    for model_dir in results_dir.iterdir():
        if model_dir.is_dir() and model_dir.name != '0real-world' and not model_dir.name.startswith('.'):
            model_path = model_dir / 'README.md'
            if model_path.exists():
                print(f"Analyzing {model_dir.name}...")
                
                # Read model predictions
                with open(model_path) as f:
                    model_content = f.read()
                model_standings = parse_markdown_standings(model_content)
                
                # Analyze each stage
                model_results = {
                    'stage1': {'correct': 0, 'teams': set()},
                    'stage2': {'correct': 0, 'teams': set()},
                    'stage3': {'correct': 0, 'teams': set()},
                    'champion': {'correct': False, 'predicted': ''}
                }
                
                # Check advancing teams predictions
                for stage in ['stage1', 'stage2', 'stage3']:
                    predicted_teams = get_advancing_teams(model_standings[stage])
                    correct_teams = predicted_teams & real_advancing[stage]
                    model_results[stage]['correct'] = len(correct_teams)
                    model_results[stage]['teams'] = predicted_teams
                
                # Check champion prediction
                playoffs_data = model_standings['playoffs']
                predicted_champion = next(team['team'] for team in playoffs_data if team['wins'] == 3)
                model_results['champion']['correct'] = (predicted_champion == real_champion)
                model_results['champion']['predicted'] = predicted_champion
                
                results[model_dir.name] = model_results
    
    # Generate report
    report = ["# CS2 Match Prediction Advancement Analysis\n\n"]
    
    # Summary table
    report.append("## Summary\n\n")
    report.append("| Model | Stage 1 | Stage 2 | Stage 3 | Champion Prediction |\n")
    report.append("|-------|----------|----------|----------|--------------------|")
    
    for model, data in results.items():
        stage1_pct = (data['stage1']['correct'] / 8) * 100
        stage2_pct = (data['stage2']['correct'] / 8) * 100
        stage3_pct = (data['stage3']['correct'] / 8) * 100
        champion = "✓" if data['champion']['correct'] else f"✗ ({data['champion']['predicted']})"
        
        report.append(f"| {model} | {stage1_pct:.1f}% | {stage2_pct:.1f}% | {stage3_pct:.1f}% | {champion} |")
    
    report.append("\n\n## Detailed Analysis\n")
    
    # Detailed analysis for each model
    for model, data in results.items():
        report.append(f"\n### {model}\n")
        
        for stage in ['stage1', 'stage2', 'stage3']:
            correct = data[stage]['correct']
            predicted = data[stage]['teams']
            actual = real_advancing[stage]
            
            report.append(f"\n#### {stage}")
            report.append(f"- Correctly predicted {correct}/8 teams ({(correct/8)*100:.1f}%)")
            report.append("- Missed predictions:")
            report.append("  - Should have advanced: " + ", ".join(sorted(actual - predicted)))
            report.append("  - Should not have advanced: " + ", ".join(sorted(predicted - actual)))
        
        report.append(f"\n#### Playoffs")
        report.append(f"- Predicted champion: {data['champion']['predicted']}")
        report.append(f"- Actual champion: {real_champion}")
        report.append(f"- Prediction {'correct' if data['champion']['correct'] else 'incorrect'}")
        
        report.append("\n---\n")
    
    # Save report
    with open(analysis_dir / 'advancement-analysis.md', 'w') as f:
        f.write('\n'.join(report))
    
    print(f"Analysis complete. Results saved in {analysis_dir}/analyze-advancements.md")
    return results

# Run the analysis
advancement_results = analyze_advancement_predictions()

FileNotFoundError: [Errno 2] No such file or directory

# Data Viz

which charts can we show based on the above analysis?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def create_visualizations(results: Dict, analysis_dir: Path):
    # Convert results to DataFrame for easier plotting
    stage_data = []
    for model, data in results.items():
        for stage in ['stage1', 'stage2', 'stage3']:
            stage_data.append({
                'Model': model,
                'Stage': stage.replace('stage', 'Stage '),
                'Accuracy': (data[stage]['correct'] / 8) * 100
            })
    df = pd.DataFrame(stage_data)

    # 1. Heatmap with original color scaling
    plt.figure(figsize=(12, 7))
    accuracy_matrix = df.pivot(index='Model', columns='Stage', values='Accuracy')
    
    sns.heatmap(accuracy_matrix, 
                annot=True, 
                fmt='.1f',
                cmap='RdYlGn',
                center=50)
    
    # Add % to annotations
    for t in plt.gca().texts:
        t.set_text(t.get_text() + '%')
    
    plt.title('Team Advancement Prediction Accuracy by Model and Stage', pad=20, size=14)
    plt.tight_layout()
    plt.savefig(analysis_dir / 'accuracy_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Bar plot
    plt.figure(figsize=(12, 7))
    overall_accuracy = df.groupby('Model')['Accuracy'].mean().sort_values(ascending=False)
    ax = overall_accuracy.plot(kind='bar', color=sns.color_palette('viridis', len(overall_accuracy)))
    plt.title('Average Team Advancement Prediction Accuracy Across All Stages', pad=20, size=14)
    plt.ylabel('Average Accuracy (%)')
    plt.xlabel('Model')
    plt.grid(True, alpha=0.3)
    
    # Add value labels on top of bars
    for i, v in enumerate(overall_accuracy):
        ax.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(analysis_dir / 'overall_performance.png', dpi=300, bbox_inches='tight')
    plt.close()

def add_visualization_section(report: List[str]):
    report.append("\n## Visualizations\n\n")
    report.append("### Accuracy Heatmap\n")
    report.append("The heatmap below shows the accuracy percentage for each model across different stages. "+
                 "Green indicates higher accuracy, while red indicates lower accuracy.\n\n")
    report.append("![Accuracy Heatmap](accuracy_heatmap.png)\n\n")
    report.append("### Overall Model Performance\n")
    report.append("Average prediction accuracy across all stages for each model:\n\n")
    report.append("![Overall Performance](overall_performance.png)\n\n")
current_dir = Path.cwd()
if current_dir.name == 'analysis':
    results_dir = current_dir.parent
else:
    results_dir = current_dir
analysis_dir = results_dir / 'analysis'
analysis_dir.mkdir(exist_ok=True)
create_visualizations(advancement_results, analysis_dir)
